### BiRealNet
This notebook demonstrates the binarization of ResNet Model using the BiRealNet

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="6"

In [3]:
import sys
import os
import shutil
sys.path.append("../../")
sys.path.append("../")

In [4]:
# load important packages
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import transforms
from trailmet.datasets.classification import DatasetFactory
from datasets import DataManager

/opt/conda/envs/ml/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/envs/ml/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c105ErrorC2ENS_14SourceLocationESs
  warn(f"Failed to load image Python extension: {e}")


###  Define Model and Model Compression Strategy

In [5]:
from trailmet.algorithms.binarize.birealnet import BirealNet
from trailmet.models.resnet import make_birealnet50

In [6]:
data_root = './'

In [7]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')
    
set_seed(1024)  

> SEEDING DONE


### Augmentations

In [8]:
crop_scale = 0.08
lighting_param = 0.1
train_transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.RandomResizedCrop(224, scale=(crop_scale, 1.0)),
                                transforms.RandomHorizontalFlip()])
val_transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Resize(256),
                                transforms.CenterCrop(224)
                            ])
test_transform = val_transform

### Configuration Class

In [9]:
class CFG:
    def __init__(self):
        self.batch_size = 64
        self.valid_size = 0.1
        self.num_train = 0
        self.epochs = 256
        self.optimizer = torch.optim.Adam
        self.lr = 0.001
        self.momentum = 0.9
        self.weight_decay = 0
        self.save_path = './save_path_resnet50_cifar100_fullbin'
        self.data_path = ''
        self.label_smooth = 0.1
        self.workers = 4
        self.device = 'cuda'
        self.dataset = 'c100'  #valid options are c100(CIFAR100), c10(CIFAR10) and tin(TinyImageNet)
        self.num_classes = 100 #10 for CIFAR10 // 100 for CIFAR100 // 200 for Tiny ImageNet 
        self.num_fp = 0

### Define DataLoaders

In [10]:
args = CFG()
args = args.__dict__
data_object = DataManager(args)
trainloader, valloader, testloader = data_object.prepare_data(train_transform, val_transform)
dataloaders = {
        'train': trainloader, 'val': valloader, "test": testloader
}

... Preparing data ...
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
using fixed split
45000 5000


### Model

In [11]:
model = make_birealnet50(num_classes=args['num_classes'], insize=32, num_fp=args['num_fp'])

In [12]:
algo = BirealNet(model, dataloaders, **args)

### Training and Evaluating the Binarized ResNet Model

In [13]:
algo.binarize()

10/05 03:56:08 PM CFG = {'batch_size': 64, 'valid_size': 0.1, 'num_train': 0, 'epochs': 256, 'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.001, 'momentum': 0.9, 'weight_decay': 0, 'save_path': './save_path_resnet50_cifar100_fullbin', 'data_path': '', 'label_smooth': 0.1, 'workers': 4, 'device': 'cuda', 'dataset': 'c100', 'num_classes': 100, 'num_fp': 0}
10/05 03:56:08 PM BirealNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activ): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BinaryBasicBlock(
      (activation): BinaryActivation()
      (conv): HardBinaryConv()
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BinaryBasicBlock(
      (activation): BinaryActivation()
      (conv): HardBinaryCo

/opt/conda/envs/ml/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch: [255][  0/704]	Time 11.089 (11.089)	Data  0.948 ( 0.948)	Loss 1.6747e+00 (1.6747e+00)	Acc@1  79.69 ( 79.69)	Acc@5  92.19 ( 92.19)
Epoch: [255][  1/704]	Time  0.321 ( 5.705)	Data  0.035 ( 0.491)	Loss 1.7187e+00 (1.6967e+00)	Acc@1  71.88 ( 75.78)	Acc@5  92.19 ( 92.19)
Epoch: [255][  2/704]	Time  0.385 ( 3.931)	Data  0.002 ( 0.328)	Loss 1.8665e+00 (1.7533e+00)	Acc@1  73.44 ( 75.00)	Acc@5  90.62 ( 91.67)
Epoch: [255][  3/704]	Time  0.424 ( 3.055)	Data  0.002 ( 0.246)	Loss 1.7047e+00 (1.7411e+00)	Acc@1  79.69 ( 76.17)	Acc@5  93.75 ( 92.19)
Epoch: [255][  4/704]	Time  0.443 ( 2.532)	Data  0.009 ( 0.199)	Loss 1.9817e+00 (1.7892e+00)	Acc@1  57.81 ( 72.50)	Acc@5  87.50 ( 91.25)
Epoch: [255][  5/704]	Time  0.433 ( 2.182)	Data  0.016 ( 0.168)	Loss 1.5817e+00 (1.7546e+00)	Acc@1  81.25 ( 73.96)	Acc@5  95.31 ( 91.93)
Epoch: [255][  6/704]	Time  0.445 ( 1.934)	Data  0.001 ( 0.144)	Loss 2.0541e+00 (1.7974e+00)	Acc@1  67.19 ( 72.99)	Acc@5  87.50 ( 91.29)
Epoch: [255][  7/704]	Time  0.488 ( 1.753

### Loading the Checkpoint to know the test accuracy

In [ ]:
chk=torch.load('./save_path_resnet50_cifar100_fullbin/c100-model_best.pth.tar')

In [21]:
chk['epoch']

250

### Test Accuracy

In [22]:
chk['best_top1_acc']

tensor(60.9000)